<a href="https://colab.research.google.com/github/JHyunjun/torch_Generative-Adversarial-Network/blob/main/LSTM_tf2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Created by Hyunjun, JANG
# limitsinx.tistory.com

import numpy as np
import pandas as pd
import tensorflow as tf
import pandas_datareader.data as pdr
import matplotlib.pyplot as plt
import datetime
tf.random.set_seed(1)

#length = 1800

#Data Load
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Batt/train_v.txt')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Batt/test_v.txt')
train_data = df_train.iloc[:,:].values
test_data = df_test.iloc[:,:].values
print(train_data)
print(test_data)

trainSet = train_data
testSet = test_data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Normalization

#print("Before Norm : ",train_data_input.shape, train_data_input)

def origin_minmax(data) : 
  arr_max = np.zeros(data.shape[1])
  arr_min = np.zeros(data.shape[1])
  print(arr_min)
  print(data.shape[1])
  for i in range(data.shape[1]) : 
    #print(i)
    max = np.max(data[:,i])
    min = np.min(data[:,i]) 
    arr_max[i] = max
    arr_min[i] = min
    pass
  return arr_max, arr_min

arr_max1, arr_min1 = origin_minmax(trainSet)
arr_max2, arr_min2 = origin_minmax(testSet)
#print(arr_max1, arr_min1)

def MinMaxScaler(data):
  for i in range(data.shape[1]) : 
    max = np.max(trainSet[:,i])
    min = np.min(trainSet[:,i])
    data[:,i] = (data[:,i] - min) / (max - min + 1e-7)
    pass
  return data

testSet = MinMaxScaler(testSet)
trainSet = MinMaxScaler(trainSet)

def back_MinMax(data,max,min):
  #print(max)
  #print(min)
  for i in range(data.shape[1]) : 
    data[:,i] = data[:,i] * (max[i] - min[i]) + min[i]
    pass
  return data

print("trainSet : ",arr_max1, arr_min1)
print("testSet : ",arr_max2, arr_min2)
print("trainSet : ",trainSet)
print("testSet : ",testSet)

In [ ]:
#Hyper parameters
seqLength = 10
dataDim = df_test.shape[1]
outputDim = 1
hiddenDim = 10
lr = 0.001
iterations = 20
batch_size = 32
'''
trainSize = int(len(train_data))
trainSet = train_data[0:trainSize]
testSize = int(len(test_data))
testSet = test_data[0:testSize]
testSet_for_result = testSet

trainSet_max = np.max(trainSet)
trainSet_min = np.min(trainSet)
testSet_max = np.max(testSet)
testSet_min = np.min(testSet)
trainSet_max = (trainSet_max[dataDim-1])
trainSet_min = (trainSet_min[dataDim-1])
testSet_max = (testSet_max[dataDim-1])
testSet_min = (testSet_min[dataDim-1])


trainSet = MinMaxScaler(trainSet)
testSet = MinMaxScaler(testSet)

print(trainSet.shape) # (5807,5)
print(testSet.shape) # (10882,5)
'''

In [ ]:
#Data Windowing
def sliding_window(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range(0, time_series.shape[0] - seq_length): 
        #print(i)
        _x = time_series[i:i + seq_length, : -1 ]
        _y = time_series[i + seq_length, [-1]]  
        print(_x, "->", _y)
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

trainX, trainY = sliding_window(trainSet, seqLength)
testX, testY = sliding_window(testSet, seqLength)


In [ ]:
# Stacked LSTM Modeling
# Return Sequences is very important for stacking
# This means weather output is Sequence of this layer or all of sequences
# Activation function "tanh" : Glorot normal initializer
# Activation function "reLU" : He Initiailizer
from tensorflow import keras
from tensorflow.keras import layers

initializer = tf.keras.initializers.GlorotNormal()
input_dim = trainSet.shape[1]-1
model = keras.Sequential()
model.add(layers.LSTM(units = hiddenDim, activation='linear',input_shape = (seqLength,input_dim), return_sequences = True, kernel_initializer = initializer))
model.add(layers.LSTM(units = hiddenDim, activation = 'linear',return_sequences = True, kernel_initializer = initializer))
model.add(layers.LSTM(units = hiddenDim, activation = 'linear',return_sequences = True, kernel_initializer = initializer))
model.add(layers.LSTM(units = hiddenDim, activation = 'linear',return_sequences = False, kernel_initializer = initializer))
model.add(layers.Dense(outputDim, activation='linear'))
adam = keras.optimizers.Adam(lr = lr)
model.compile(optimizer=adam, loss='mse')
model.summary()


In [ ]:
# Training
hist = model.fit(trainX, trainY, epochs = iterations, batch_size = batch_size)

In [ ]:
# Evaluate Testset
res = model.evaluate(testX, testY, batch_size = batch_size)
print(res)
#res_batch_size1 = model.evaluate(testX, testY, batch_size = 1) #Very Slow


In [ ]:
# Estimating
xhat = trainX
yhat = model.predict(xhat)
#print(testY)
print(yhat)

#print("Evaluate : {}".format(np.sqrt(np.average((yhat - testY)**2))))

In [ ]:
#Estimating
maxs = np.max(trainSet[:,-1])
mins = np.min(trainSet[:,-1])
print(maxs,mins)
estimated = yhat*(maxs-mins) + mins
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Batt/train_v.txt')
testSet_for_result = df_train
real = testSet_for_result.iloc[:testSet_for_result.shape[0]-seqLength,-1] # estimated + seqLength
print(estimated)

In [ ]:
plt.figure()
time1 = 0
time2 = 300
plt.plot(estimated[time1:time2],label = "Estimated by Stacked LSTM")
plt.plot(real[time1:time2], label = "Real")
plt.legend(prop = {'size':10})
print(estimated)

plt.figure()
plt.plot(estimated[time1:time2],label = "Estimated by Stacked LSTM")
plt.legend(prop = {'size':10})

plt.figure()
plt.plot(real[time1:time2], label = "Real")
plt.legend(prop = {'size':10})


In [ ]:
#Generated by GAN data

gan1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/GAN/pytorch/Timeseries-GAN/proof/220330/220330_GEN_IntegralCAP.txt')
gan2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/GAN/pytorch/Timeseries-GAN/proof/220330/220330_GEN_genCAP.txt')
print(gan1.shape)

gan1 = gan1.values
gan2 = gan2.values

df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Batt/train_v.txt')
trainSet = df_train.values

soc_arrange = 0

gan1[:,2] = gan1[:,2] + soc_arrange
gan2[:,2] = gan2[:,2] + soc_arrange

def MinMaxScaler(data):
  for i in range(data.shape[1]) : 
    max = np.max(trainSet[:,i])
    min = np.min(trainSet[:,i])
    data[:,i] = (data[:,i] - min) / (max - min + 1e-7)
    pass
  return data

#Data Windowing
def sliding_window(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range(0, time_series.shape[0] - seq_length): 
        #print(i)
        _x = time_series[i:i + seq_length, : -1 ]
        _y = time_series[i + seq_length, [-1]]  
        #print(_x, "->", _y)
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

gan1 = MinMaxScaler(gan1)
gan2 = MinMaxScaler(gan2)

#print(gan1)

trainX_gan1, trainY_gan1 = sliding_window(gan1, seqLength)
trainX_gan2, trainY_gan2 = sliding_window(gan2, seqLength)

In [ ]:
yhat_gan1 = model.predict(trainX_gan1)
yhat_gan2 = model.predict(trainX_gan2)

In [ ]:
print(maxs,mins)
yhat_gan1_denorm = yhat_gan1*(233.3) + mins
yhat_gan2_denorm = yhat_gan2*(233.3) + mins

trainY_gan1_denorm = trainY_gan1*(maxs-mins) + mins
trainY_gan2_denorm = trainY_gan2*(maxs-mins) + mins
#print("1 : ",trainY_gan1_denorm)
#print("2 : ",trainY_gan2_denorm)

In [ ]:
real1 = trainY_gan1_denorm
real2 = trainY_gan2_denorm
plt.figure()
plt.plot(yhat_gan1_denorm,label = "GAN1_Integral CAP")
plt.plot(real1, label = "Generated_Volt")
plt.legend(prop = {'size':10})

plt.figure()
error = real1

plt.figure()
plt.plot(yhat_gan2_denorm,label = "GAN2_gen CAP")
plt.plot(real2, label = "Generated Volt")
plt.legend(prop = {'size':10})

In [ ]:
print(yhat_gan1_denorm.shape)
print(trainY_gan1_denorm.shape)